In [1]:
from meta.data_processor import DataProcessor

ticker_list = ['ETHUSDT', 'BTCUSDT']

TIME_INTERVAL = '1h'

TRAIN_START_DATE = '2015-01-01'
TRAIN_END_DATE = '2019-08-01'
TRADE_START_DATE = '2019-08-01'
TRADE_END_DATE = '2020-01-03'

if_vix = False

p = DataProcessor(data_source='binance', start_date=TRAIN_START_DATE,
                  end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL)
p.download_data(ticker_list=ticker_list)
p.clean_data()
df = p.dataframe

binance successfully connected
Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (41494, 8)


In [2]:
import pandas as pd
from talib.abstract import MACD, RSI, CCI, DX

df = pd.read_csv('data/dataset.csv')
technical_indicator_list = ['macd',
                            'macd_signal',
                            'macd_hist',
                            'rsi',
                            'cci',
                            'dx'
                            ]


def add_technical_indicator(df, tech_indicator_list):
  final_df = pd.DataFrame()
  for i in df.tic.unique():
    tic_df = df[df.tic == i].copy()
    tic_df['rsi'] = RSI(tic_df['close'], timeperiod=14)
    tic_df['macd'], tic_df['macd_signal'], tic_df['macd_hist'] = MACD(
      tic_df['close'], fastperiod=12,
      slowperiod=26, signalperiod=9)
    tic_df['cci'] = CCI(tic_df['high'], tic_df['low'], tic_df['close'],
                        timeperiod=14)
    tic_df['dx'] = DX(tic_df['high'], tic_df['low'], tic_df['close'],
                      timeperiod=14)
    final_df = pd.concat([final_df, tic_df])
  return final_df


processed_df = add_technical_indicator(df, technical_indicator_list)
processed_df.index = pd.to_datetime(processed_df.time)
processed_df.drop('time', inplace=True, axis=1)
processed_df.tail()

,open,high,low,close,volume,adjusted_close,tic,rsi,macd,macd_signal,macd_hist,cci,dx
time,,,,,,,,,,,,,
2020-01-02 19:00:00,7162.18,7180.00,7153.33,7161.83,1446.219984,7161.83,BTCUSDT,44.288677,-19.937711,-17.543580,-2.394131,-4.322773,20.017467
2020-01-02 20:00:00,7161.89,7168.67,7139.03,7139.79,761.156570,7139.79,BTCUSDT,39.603234,-20.237987,-18.082462,-2.155525,-32.074422,27.022707
2020-01-02 21:00:00,7139.73,7163.40,7139.03,7158.29,794.030497,7158.29,BTCUSDT,44.874947,-18.766830,-18.219335,-0.547494,-11.448699,27.022707
2020-01-02 22:00:00,7158.86,7163.35,7107.43,7131.15,1566.280693,7131.15,BTCUSDT,39.436672,-19.565361,-18.488541,-1.076821,-69.799501,40.400660
2020-01-02 23:00:00,7131.18,7157.10,7122.34,7135.59,956.300925,7135.59,BTCUSDT,40.702719,-19.613836,-18.713600,-0.900236,-46.120119,40.400660
